In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

### load dataset

In [1]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")
dataset = dataset.select(list(range(1000)))
# dataset['text']

Repo card metadata block was not found. Setting CardData to empty.


### load model

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

### answer before finetuning

In [14]:
prompt = """
The following is a friendly conversation between a human and an AI. The AI is
talkative and provides lots of specific details from its context.
 
Current conversation:
 
Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.
AI:
""".strip()
 
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)
print(input_ids)

tensor([[  487,  1863,   304,   241,  6350,  5501,  1192,   241,  2079,   273,
           267,  8317,    25,   390,  8317,   304,   193, 22685,   998,   273,
          2665,  4587,   275,  2005,  2861,   427,   701,  4436,    25,  1212,
           193,  9797,  5501,    37,  1212,   193, 23431,    37,  1853,   299,
          3131,   241,  1866,  9705,   544,   248, 23335,   275,   248,  1650,
           204,    13,  3032,   387,  1799,   100,    13,   272, 17750,    42,
          4012,   745,  6432,  3236,   271,  2879,   988,   387,  1799,   424,
           272,   248, 17170,  1208,   273, 27421,  5228,  1959,    25,   193,
         17362,    37]], device='cuda:0')


In [15]:
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config
with torch.inference_mode():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
    )

In [16]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

The following is a friendly conversation between a human and an AI. The AI is
talkative and provides lots of specific details from its context.
 
Current conversation:
 
Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.
AI: Sure, I'd be happy to help. Monopsony refers to a situation where one party has significant power over another in a particular market or industry. In the context of the labour market, it can occur when there are few firms competing for workers, leading to high wages and poor working conditions. Examples of this include industries such as tech and finance, where companies like Google and Amazon have large amounts of bargaining power over their employees. This power allows them to set salaries and benefits that may not align with fair labor standards. Research suggests that the prevalence of monopsony in these sectors 

### model fine tune

In [17]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [18]:
from transformers import TrainingArguments

output_dir = "./ziming_results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4 #4*4=16
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 2
learning_rate = 2e-4 # if 4 gpu, decrease the lr
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [24]:
from trl import SFTTrainer
# max_seq_length = 512
max_seq_length = 215

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [25]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [26]:
%env WANDB_CONFIG_DIR="/home/ac.zyang/wandb"
trainer.train()

env: WANDB_CONFIG_DIR="/home/ac.zyang/wandb"


/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
2,1.931100
4,2.180900
6,1.921100
8,1.777200
10,1.943000
12,1.931000
14,1.606100
16,1.904900
18,1.615600
20,1.721800


/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarnin

TrainOutput(global_step=100, training_loss=1.3285811936855316, metrics={'train_runtime': 9868.018, 'train_samples_per_second': 0.649, 'train_steps_per_second': 0.01, 'total_flos': 5.020864090186752e+16, 'train_loss': 1.3285811936855316, 'epoch': 6.35})

In [27]:
trainer.save_model('fine_tuned_model')

In [28]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
tuned_model = AutoModelForCausalLM.from_pretrained('fine_tuned_model')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
tuned_model = tuned_model.eval()
 
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [30]:
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 256,
  "pad_token_id": 11,
  "repetition_penalty": 1.7,
  "temperature": 0,
  "use_cache": false
}

### answer after finetuning

In [31]:
prompt = """
The following is a friendly conversation between a human and an AI. The AI is
talkative and provides lots of specific details from its context.
 
Current conversation:
 
Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.
AI:
""".strip()
 
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)
print(input_ids)

tensor([[  487,  1863,   304,   241,  6350,  5501,  1192,   241,  2079,   273,
           267,  8317,    25,   390,  8317,   304,   193, 22685,   998,   273,
          2665,  4587,   275,  2005,  2861,   427,   701,  4436,    25,  1212,
           193,  9797,  5501,    37,  1212,   193, 23431,    37,  1853,   299,
          3131,   241,  1866,  9705,   544,   248, 23335,   275,   248,  1650,
           204,    13,  3032,   387,  1799,   100,    13,   272, 17750,    42,
          4012,   745,  6432,  3236,   271,  2879,   988,   387,  1799,   424,
           272,   248, 17170,  1208,   273, 27421,  5228,  1959,    25,   193,
         17362,    37]], device='cuda:0')


In [32]:
with torch.inference_mode():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
    )

/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ac.zyang/miniconda3/envs/ke/lib/python3.11/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [33]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

The following is a friendly conversation between a human and an AI. The AI is
talkative and provides lots of specific details from its context.
 
Current conversation:
 
Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.
AI: Sure, I'd be happy to help! A monopoly exists when there are few or no competitors for a particular good or service. Monopolistic markets occur when a single firm dominates the industry. In terms of labor markets, a monopolist would have considerable power over workers seeking employment in that field. 

A possible example of a potential monopoly in the labor market could be artificial intelligence replacing large parts of manual workforces. If one company were responsible for developing and deploying such technology, they may set wages and working conditions with little competition. This situation might lead to lowe

In [3]:
# answer in training dataset
standard_answer = """
 "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. 
 In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. 
 The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.
 Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). 
 In these industries, workers often face low wages, limited benefits, and reduced bargaining power, 
 leading to a situation where they are dependent on the employer for their livelihood. This dependence can result 
 in further suppression of wages and a decline in working conditions.\n\nOverall, the concept of monopsony is 
 essential to understanding the dynamics of labor markets and the impact of market power on workers. 
 Further research is needed to understand the extent and impact of monopsonies on the economy and to develop 
 policies to address this issue.\n\nReferences:\nBivens, J., & Mishel, L. (2013). The Pay of Corporate Executives 
 and Financial Professionals as Evidence of Rents in Top 1 Percent Incomes. Journal of Economic Perspectives, 
 27(3), 57-78.
"""
standard_answer

'\n "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. \n In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. \n The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). \n In these industries, workers often face low wages, limited benefits, and reduced bargaining power, \n leading to a situation where they are dependent on the employer for their livelihood. This dependence can result \n in further suppression of wages and a decline in working conditions.\n\nOverall, the concept o